# Imports

In [1]:
import sys
import logging
import shutil
from uuid import uuid4
from tempfile import gettempdir
from os.path import join, isdir
import folium
from aequilibrae import Parameters, logger, Project

In [2]:
from building_parameters import model_place
from country_borders import get_country_borders

# Environment setup

In [3]:
stdout_handler = logging.StreamHandler(sys.stdout)
formatter = logging.Formatter("%(asctime)s;%(name)s;%(levelname)s ; %(message)s")
stdout_handler.setFormatter(formatter)
logger.addHandler(stdout_handler)

In [4]:
fldr = f'../{model_place}'

if isdir(fldr):
  shutil.rmtree(fldr)

# Create Road Network from OSM

In [5]:
project = Project()
project.new(fldr)

/srv/conda/envs/saturn/lib/python3.9/site-packages/aequilibrae/parameters.py:53: UserWarning: No pre-existing parameter file exists for this project. Will use default
  warn('No pre-existing parameter file exists for this project. Will use default')


2022-05-06 09:46:46,812;aequilibrae;INFO ; Created project on ../Monaco


In [6]:
new_fields = [{'bridge': {'description': 'bridge flag', 'osm_source': 'bridge', 'type': 'text'}},
              {'tunnel': {'description': 'tunnel flag', 'osm_source': 'tunnel', 'type': 'text'}},
              {'toll': {'description': 'toll flag', 'osm_source': 'toll', 'type': 'text'}},
              {'surface': {'description': 'pavement surface', 'osm_source': 'surface', 'type': 'text'}}]

par = Parameters()
par.parameters['network']['links']['fields']['one-way'].extend(new_fields)
par.write_back()

In [7]:
%%time
project.network.create_from_osm(place_name=model_place)

2022-05-06 09:46:48,283;aequilibrae;INFO ; PLACE FOUND:Monaco
2022-05-06 09:46:48,327;aequilibrae;INFO ; Downloading data
2022-05-06 09:46:48,408;aequilibrae;DEBUG ; Downloading polygon 1 of 1
2022-05-06 09:46:58,414;aequilibrae;DEBUG ; Posting to http://overpass-api.de/api/interpreter with timeout=540, "{'data': '[out:json][timeout:540];(way["highway"]["area"!~"yes"]["highway"!~"proposed|raceway|construction|abandoned|platform"]["service"!~"parking|parking_aisle|driveway|private|emergency_access"]["access"!~"private"](43.724760,7.409028,43.751931,7.439870);>;);out;'}"
2022-05-06 09:47:01,129;aequilibrae;INFO ; Downloaded 1,994.6KB from overpass-api.de in 2.72 seconds
2022-05-06 09:47:01,262;aequilibrae;INFO ; Building Network
2022-05-06 09:47:01,382;aequilibrae;INFO ; Separating nodes and links
2022-05-06 09:47:01,530;aequilibrae;INFO ; Setting data structures for nodes
2022-05-06 09:47:01,593;aequilibrae;INFO ; Setting data structures for links
2022-05-06 09:47:01,606;aequilibrae;INF

# Restricts import to the country's borders only

In [8]:
place_geo = get_country_borders(model_place)

In [9]:
project.conn.execute('CREATE TABLE IF NOT EXISTS country_borders("country_name" TEXT);')
project.conn.execute("SELECT AddGeometryColumn( 'country_borders', 'geometry', 4326, 'MULTIPOLYGON', 'XY' );")

project.conn.execute("SELECT CreateSpatialIndex( 'country_borders' , 'geometry' );")
project.conn.commit()
project.conn.execute('INSERT into country_borders(country_name, geometry) VALUES(?, GeomFromWKB(?, 4326));', [model_place, place_geo.wkb])
project.conn.commit()

In [10]:
%%time
sql = """DELETE from Links where link_id not in (SELECT a.link_id
FROM links AS a, country_borders as b
WHERE ST_Intersects(a.geometry, b.geometry) = 1)"""

project.conn.execute(sql)
project.conn.commit()

GEOS error: TopologyException: side location conflict at 7.4120416000000002 43.728095500000002


CPU times: user 352 ms, sys: 17.3 ms, total: 369 ms
Wall time: 423 ms


In [11]:
project.close()

2022-05-06 09:47:25,700;aequilibrae;INFO ; Closed project on ../Monaco
